In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib as mpl

label_size = 14
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size

In [ ]:
data = np.array([0,3,4,5])
conv1d_filter = np.array([1,2])

In [ ]:
def Conv1D_Numpy(Seq, Kernel):
    kernel_size = len(Kernel)
    Length = len(Seq)

    output = []
    for i in range(Length-kernel_size+1):
        conv = np.dot(Seq[i:i+kernel_size], Kernel)
        print(Seq[i:i+kernel_size], "*", Kernel, "=> ", conv)

        output.append(conv)

    output = np.array(output)

    return output

In [ ]:

output = Conv1D_Numpy(data, conv1d_filter)
output

In [ ]:
# # 1D 컨볼루션 모델을 생성하고 컴파일하는 역할을 합니다.
def Conv1D_compile(n_filters, SequenceLength, n_features):
    conv_model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=n_filters, # 컨볼루션 레이어에서 사용할 필터(커널)의 수입니다.
                               kernel_size=2,
                               strides=1,
                               padding='valid',
                               input_shape=(SequenceLength, n_features), # 입력 시퀀스의 길이와 입력 시퀀스의 특성(또는 차원) 수입니다
                               use_bias=False, name='c1d')])

    conv_model.compile(loss=tf.losses.MeanAbsoluteError(),
                       optimizer=tf.optimizers.Adam(learning_rate=5e-2))

    conv_model.summary()

    return conv_model


# 모델을 학습시키고, 특정 에폭마다 모델의 가중치 변화를 추적하여 시각화하는 역할을 합니다.
def Conv1D_Fit_and_PlotWeights(model, X, y, epochs, n_weights, freq=20):
    w, loss, mae = [], [], []

    for r in range(epochs):
        history = model.fit(X, y, verbose=0)
        if r%freq==0:

            w.append(np.sort(model.layers[0].get_weights()[0].reshape(n_weights)))  # 초기 무작위 가중치.
            loss.append(history.history['loss'][0])

    w = np.array(w)

    fig, ax = plt.subplots(figsize=(8,4))

    epoch = np.arange(0,len(w))*20 # 매 20번째 epochs마다 모델의 가중치를 추출하고 기록

    for n in range(n_weights):
        label = "w_{} -> {}".format(n, n+1)
        plt.plot(epoch,w[:,n], label=label, linewidth=3)
        ax.axhline(n+1, c='gray', linestyle='--')

    plt.xlabel("epoch", fontsize=14)
    plt.ylabel("weights", fontsize=14)
    plt.legend(loc='upper left', bbox_to_anchor=(1., 1.01), fontsize=14)
    plt.show()

In [ ]:
X = data.reshape((1, data.shape[0], 1)) # -> np.array([0,3,4,5]).reshape(1,4,1)과 동일
y = output.reshape((1, output.shape[0], 1)) # -> np.array([6,11,14]).reshape(1,3,1)과 동일

In [ ]:
model_cnn = Conv1D_compile(n_filters=1, SequenceLength=4, n_features=1)

In [ ]:
Conv1D_Fit_and_PlotWeights(model=model_cnn, X=X, y=y,
                                     epochs=500, n_weights=2)

### 음성 데이터 1D Convolution 적용 예제

In [ ]:
import torch
import torch.nn as nn

# 무작위 데이터 생성
batch_size, channels, length = 1, 1, 8000  # 1초 분량의 샘플링 레이트를 8000HZ
x_pytorch = torch.randn(batch_size, channels, length)

# Conv1D 층 정의
conv1d_pytorch = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1) # 16개의 채널 추가 생성

# Conv1D 적용
output_pytorch = conv1d_pytorch(x_pytorch)

print("음성 데이터 출력 형태:", x_pytorch.shape)
print("----------------------------------------------------")
print("PyTorch Conv1D 출력 형태:", output_pytorch.shape)

In [ ]:
import tensorflow as tf

# 무작위 데이터 생성
x_tensorflow = tf.random.normal((batch_size, length, channels))

# Conv1D 층 정의 및 적용
conv1d_tensorflow = tf.keras.layers.Conv1D(filters=16, kernel_size=3, strides=1, padding="same")
output_tensorflow = conv1d_tensorflow(x_tensorflow)

print("음성 데이터 출력 형태:", x_tensorflow.shape)
print("----------------------------------------------------")
print("TensorFlow Conv1D 출력 형태:", output_tensorflow.shape)

### 실제 음성 데이터 멜 스펙토그램 변환

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

file_path = './213_003_1219.wav'

# 10초 길이의 8000Hz 샘플링 레이트 음성 데이터 로드
y, sr = librosa.load(file_path, sr=8000, duration=12)

# 멜 스펙트로그램을 계산합니다.
mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, hop_length=256, n_fft=512)
mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

# 멜 스펙트로그램의 형태를 확인합니다.
print(mel_spectrogram_db.shape)

# 멜 스펙트로그램을 시각적으로 그립니다.
plt.figure(figsize=(10, 4))
librosa.display.specshow(mel_spectrogram_db, sr=sr, hop_length=256, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram')
plt.tight_layout()
plt.show()

### 멜 스펙토그램에 conv 1d 적용

In [ ]:
mel_spectrogram_tensor = torch.tensor(mel_spectrogram_db[np.newaxis, :], dtype=torch.float)

# 컨볼루션 레이어 정의
conv1d_layer = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=3, stride=1, padding=1) # in_channels = input 데이터 채널의 수

# 레이어에 멜 스펙트로그램 텐서를 적용합니다. PyTorch는 (batch, channels, length) 형태를 입력해야 합니다.
# 우리의 데이터는 (1, 128, length) 형태로, 128은 멜 빈의 수, length는 시간 축에 대한 길이입니다.
conv_output = conv1d_layer(mel_spectrogram_tensor)

# 결과의 형태를 출력합니다.
print('Conv1d Output Shape:', conv_output.shape)

# 멜 스펙트로그램을 시각적으로 그립니다.
plt.figure(figsize=(10, 4))
librosa.display.specshow(mel_spectrogram_db, sr=sr, hop_length=256, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram')
plt.tight_layout()
plt.show()

In [ ]:
# groups 추가
conv1d_layer = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=3, stride=1, padding=1, groups=4)

# 레이어에 멜 스펙트로그램 텐서를 적용합니다.
conv_output = conv1d_layer(mel_spectrogram_tensor)

# 결과의 형태를 출력합니다.
print('Conv1d Output Shape with groups=4:', conv_output.shape)

In [ ]:
# Depthwise 컨볼루션을 위한 Conv1d 레이어 정의
# 입력 채널 수와 출력 채널 수가 동일하고, groups가 in_channels와 같습니다.
conv1d_layer_depthwise = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1, groups=128)

# 레이어에 멜 스펙트로그램 텐서를 적용합니다.
conv_output_depthwise = conv1d_layer_depthwise(mel_spectrogram_tensor)

# 결과의 형태를 출력합니다.
print('Depthwise Conv1d Output Shape:', conv_output_depthwise.shape)